# Understand Number Spaces - MinMax Mode

To enable searches with well known MAX and MIN logic `NumberSpace` creates the opportunity to embed numeric data in a searchable way.

Check Similar mode [here](number_embedding_similar.ipynb).

In [1]:
%pip install superlinked==5.4.1

In [2]:
from superlinked.framework.common.schema.id_schema_object import IdField
from superlinked.framework.common.schema.schema import schema
from superlinked.framework.common.schema.schema_object import Integer
from superlinked.framework.dsl.index.index import Index
from superlinked.framework.dsl.space.number_space import NumberSpace, Mode

from superlinked.framework.dsl.executor.in_memory.in_memory_executor import (
    InMemoryExecutor,
)
from superlinked.framework.dsl.source.in_memory_source import InMemorySource
from superlinked.framework.dsl.query.query import Query

In [3]:
@schema
class Person:
    id: IdField
    height: Integer  # set height in cms for example
    outstanding_debt_amount: Integer  # set in dollars for example


person = Person()

## Minimum mode

For preferring lower numbers

In [4]:
debt_space = NumberSpace(
    number=person.outstanding_debt_amount, min_value=0, max_value=1e6, mode=Mode.MINIMUM
)  # debts higher than max_value are effectively capped

## Maximum mode

To prefer higher numbers

In [5]:
height_space = NumberSpace(
    number=person.height, min_value=100, max_value=220, mode=Mode.MAXIMUM
)  # heights out of this range are considered an outlier in our usecase

In [6]:
person_index = Index(height_space)

In [7]:
source: InMemorySource = InMemorySource(person)
executor = InMemoryExecutor(sources=[source], indices=[person_index])
app = executor.run()

In [8]:
source.put(
    [
        {
            "id": "person1",
            "height": 175,
            "outstanding_debt_amount": 0,
        },
        {
            "id": "person2",
            "height": 180,
            "outstanding_debt_amount": 0,
        },
    ]
)

In [9]:
query = Query(person_index).find(person)

In [10]:
result = app.query(query)
result.to_pandas()

,height,outstanding_debt_amount,id
0,180,0,person2
1,175,0,person1


The person having larger height comes first, as it is expected because of the usage of a `NumberSpace` with `MAXIMUM` `Mode`. 